In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch
import pandas as pd
import math
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import requests
import json
import hashlib
from os import listdir
from os.path import isfile, join

In [4]:
encode("https://www.coles.com.au/product/the-natural-confectionary-co.-sour-squirms-lollies-220g-2351254")

'6b2f47f3127b85d25a4f6f6c407100d0'

In [2]:
es = Elasticsearch( 'http://localhost:9200')

In [3]:
INDEX_NAME = 'grocery-index'

In [37]:
def process_text(html):
    if html is None:
        return ''
    soup = BeautifulSoup(html, 'html.parser')
    node = soup.find("div", {"class": "sc-a2942c6c-3 dgpqzY coles-targeting-SectionHeaderDescription"})
    if node is None:
        return node
    return node.text.replace(u'\xa0', u' ')

In [43]:
def ingest_from_file(file_path):
    # Opening JSON file
    f = open(file_path)
    data = json.load(f)
    f.close()
    print(f'ingest from {file_path}')
    for product in data:
        prod_json = {}
        prod_json['id'] = encode(product['url'])
        prod_json['title'] = product['title']
        prod_json['source'] = 'COLES'c
        prod_json['pricetext'] = product['price']
        prod_json['price'] = float(product['price'].replace('$','')) if product['price'] is not None else None
        prod_json['packageprice'] = product['package_price']
        prod_json['productimage'] = product['product_image']
        prod_json['productdetails'] = process_text(product['product_details'])
        prod_json['url'] = product['url']
        try :
            res = es.index(index=INDEX_NAME, id=prod_json['id'], body=prod_json)
            # print(res['result'])
        except:
            print ('index error')
    print('index_completed')

In [ ]:
def main():
    data_dir = '/Users/vli/Work/RetailSearch/RetailSearch/data'
    data_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and 'product' in f]
    for file_path in data_files:
        ingest_from_file(file_path)

In [ ]:
if __name__ == "__main__":
    main()